## Linear Regression



### Causation versus Correlation



 Back in
 my home country, and before the hippy movement changed our culture,
 kids, who were curious about where the babies come from, were told that
 they are brought by the stork (a large bird, see
 Fig.[fig:storksa](#fig:storksa)). Storks were indeed a common sight in rural
 areas, and large enough to sell this story to a 3-year-old.

![img](Ringed_white_stork.png "The Stork. Image by Soloneying, from ![img](https://commons.wikimedia.org/wiki/File:Ringed_white_stork.jpg) Downloaded Nov 22<sup>nd</sup> 2019.")

Sadly, as grown-up scientists with a penchant for critical thinking,
we want to know if there is data to support this idea. Specifically,
we should see a good correlation between the number of storks and the
number of babies. Low and behold, these two variables actually
correlate in a statistically significant way. Countries with larger
stork populations have higher birthrates.  Since both variables
increase together, this is called a positive correlation. See
Fig. [4](#org408ee76)

![img](./storks.png "The birthrate and the number of stork pairs correlate in a statistical significant way. This analysis suggests that each stork pair delivers about 29 human babies, and that about 225 thousand babies were born otherwise. Data after Matthews 2000.")

Now, does this prove that the storks deliver the babies? Obviously (or so we
think) not. Just because two observable quantities correlate does in no way
imply that one is the cause of the other. The more likely explanation is that
both variables are affected by a common process (i.e., industrialization).

It is a common mistake to confuse correlation with causation. Another
good example is to correlate drinking with heart attacks. This surely
will correlate, but the story is more complex. Are there, e.g.,
patterns like drinkers tend to do less exercise than non-drinkers? So
even if you have a good hypothesis why two variables are correlated,
the correlation itself proves nothing. 

Irrespective of a causal relationship, we can express the correlation
between two datasets as the Pearson Product Moment Correlation
Coefficient (PPMC, typically called `r`) to describe the strength and direction of the
relationship between two variables.   The PPMCC (lower case r) varies
between +1 (perfect positive correlation) and -1 (perfect negative or
inverse correlation). Correlations are described as weak if they are
between +0.3 and -0.3, strong if they greater than +0.7 or less than
-0.7”. Note, that correlation analysis makes no assumptions about the
functional form between y (dependent variable) and x (independent
variable). IN other words, the PPMC says nothing about whether the
correlation is linear, logarithmic, exponential etc.

We can use the `corr()` method of the pandas series object to calculate the PPMCC



In [1]:
import pandas as pd  # inport pandas as pd
import pathlib as pl

fn: str = "storks_vs_birth_rate.csv"  # file name
cwd: pl.Path = pl.Path.cwd()
fqfn: pl.Path = pl.Path(f"{cwd}/{fn}")

# this little piece of code could have saved me 20 minutes
if not fqfn.exists():  # check if the file is actually there
    raise FileNotFoundError(f"Cannot find file {fqfn}")

df: pd.DataFrame = pd.read_csv(fn)  # read data
df.columns = ["Babies", "Storks"]  # replace colum names 
b: pd.Series = df["Babies"]
s: pd.Series = df["Storks"]

print(f" r = {s.corr(b):.2f}")

So we confirm that the number of babies and storks correlate.



### Understanding the results of a linear regression analysis



Linear regression analysis takes correlation analysis one step further
and determines how well a linear function (e.g., a straight line) can
describe the relation between x and y.  The equation of a straight
line `y = mx +b` is fitted to the scatter of data by changing `a` and
`m` in such a way that the difference between the measured data and
the model prediction is minimized.

The Coefficient of Determination, or r<sup>2</sup> expresses how well
a sloping straight line can explain the correlation between the 
dependent (y) and a single independent (x) variable. In the above
figure, r<sup>2</sup> = 0.38, which means that 38% of the new-born babies could
be explained by a linear correlation with the number of storks.

In many cases, more than one independent variable is needed to explain
the scatter in the data. In this case Multiple Linear Regression is
used where y = x1 + x2 + x3….xn + b. From this analysis
(i.e. simultaneous solving for a system of linear equations – remember
your Linear Algebra course!) the Multiple Coefficient of Determination
(R<sup>2</sup>) is used to express the amount of explained variation in y by a
combination of independent variables. By default, many programs use
the R<sup>2</sup> number even when there is only one independent variable. This
can be misleading as capital R<sup>2</sup> should be reserved for analyses that
involve multiple independent variables.

From a user perspective, we are interested to understand how good the
model actually is, and how to interpret the key indicators of a given
regression model:

-   **r<sup>2</sup>:** or coefficient of determination. \index{linear
    regression!rsquare} \index{linear regression!coefficient of
    determination} This value is in the range from zero to one and
    expresses how much of the observed variance \index{linear
    regression!variance} in the data is explained by the regression
    model. So a value of r<sup>2</sup>=0.7 indicates that 70% of the variance is
    explained by the model, and that 30% of the variance is explained
    by other processes which are not captured by the linear model
    (e.g., measurements errors, or some non-linear effect affecting `x`
    and `y`). In Fig. [BROKEN LINK: fig:storks] 38% of the variance in the birthrate
    can be explained by the increase in stork pairs.
-   **p:** When you do a linear regression, you basically state the
    hypothesis that `y` depends on `x` and that they are linked by a
    linear equation. If you test a hypothesis, you however also have to
    test the so-called **null-hypothesis**, which in this case would
    \index{linear regression!null hypothesis} state that `y` is
    \index{linear regression!p-value} unrelated to `x`. The p-value
    expresses the likelihood that the null-hypothesis is true. So a
    p-value of 0.1 indicates a 10% chance that your data does not
    correlate. A p-value of 0.01, indicates a 1% chance that your data
    is not correlated. Typically, we can reject the null-hypothesis if
    `p < 0.05`, in other words, we are 95% sure the null hypothesis is
    wrong. In Fig. [BROKEN LINK: fig:storks], we are 99.2% sure the null hypothesis is
    wrong. Note that there is not always a simple relationship between
    r<sup>2</sup> and p.



### The statsmodel library




Pythons success rests to a considerable degree on the myriad of third
party libraries which, unlike matlab, are typically free to use. In
the following, we will use the "statsmodel" library, but there are
plenty of other statistical libraries we could use as well. 

The statsmodel library provides different interfaces. Here we will use
the formula interface, which is similar to the R-formula
syntax. However, not all statsmodel functions are available through
this interface (yet?). First, we import the needed libraries:



In [1]:
import pandas as pd  # import pandas as pd
import pathlib as pl

# define the file and sheetname we want to read. Note that the file
# has to be present in the local working directory!
fn: str = "storks_vs_birth_rate.csv"  # file name
cwd: pl.Path = pl.Path.cwd()
fqfn: pl.Path = pl.Path(f"{cwd}/{fn}")
if not fqfn.exists():  # check if the file is actually there
    raise FileNotFoundError(f"Cannot find file {fqfn}")

df: pd.DataFrame = pd.read_csv(fn)  # read data
df.columns = ["Babies", "Storks"]  # replace colum names
df.head()  # test that all went well

Before we perform a regression analysis, me must test if the  data follows a normal
distribution. There are a variety of tests to check for normality, but
here we will simply use a histogram plot which either shows a bell
curve distribution or not 



In [1]:
import matplotlib.pyplot as plt

fig: plt.Figure
ax1: plt.Axes
ax2: plt.Axes

fig, [ax1, ax2] = plt.subplots(nrows=2, ncols=1)  #
fig.set_size_inches(6, 9)
ax1.hist(
    df.iloc[:, 0],
)
ax2.hist(df.iloc[:, 1])
ax1.set_title("Babies")
ax2.set_title("Storks")
plt.show()

As you can see from the histogram, our data shows anything but a
normal distribution! We will use it anyway since it is a fun
dataset. However, the above test is crucial if you ever want to do a
real regression analysis!

For the regression model, we want to analyze whether the number of
storks predicts the number of babies. In other words, does the birth
rate depend on the number of storks? For this, we need to define a
statistical model, and test whether the model predictions will fit the
data:

-   The gory details of this procedure are beyond the scope of this
    course - if you have not yet taken a stats class, I do recommend
    doing so!
-   There are many ways of doing this. Here we use an approach which
    is common in `R`

As of November 2021, there is no type hinting support for the statsmodel library.  However, in order to  distinguish between the various statsmodel object types, I use the following hints:



In [1]:
import statsmodels.formula.api as smf

model: smf.ols  # ordinary least square model object
results: smf.ols.fit  # smf.ols.model results object

After importing the data, we now create a statistical model on line 16
in the code below.  Pay attention how the model is specified with the
formula `"Babies ~ Storks`, which states that the number of Babies
should depend on the number of storks. These names must correspond to
the variable names in the dataframe `df`!

Once the model is defined, we request to fit the model against the data (line 17)
The results of the `fit()` method will be stored in the `results` variable.
Line 18, then  invokes the
`summary()` method of the results object.



In [1]:
import statsmodels.formula.api as smf
import pandas as pd  # import pandas as pd
import pathlib as pl

# define the file and sheetname we want to read. Note that the file
# has to be present in the local working directory!
fn: str = "storks_vs_birth_rate.csv"  # file name
cwd: pl.Path = pl.Path.cwd()
fqfn: pl.Path = pl.Path(f"{cwd}/{fn}")
if not fqfn.exists():  # check if the file is actually there
    raise FileNotFoundError(f"Cannot find file {fqfn}")

df: pd.DataFrame = pd.read_csv(fn)  # read data
df.columns = ["Babies", "Storks"]  # replace colum names

model: smf.ols = smf.ols(formula="Babies ~ Storks", data=df)
results: model.fit = model.fit()  # fit the model to the data
print(results.summary())  # print the results of the analysis

Plenty of information here, probably more than you asked for. 
Let's tease out the important ones:

-   The first line states that `Babies` is the dependent variable. This
    is useful and will help you to catch errors in your model
    definition.
-   The second lines confirms that this is an ordinary least squares model
-   Then, there are also a couple of warnings, indicating that your
    data quality may be less than excellent. But we knew this already
    from testing whether the data is normal distributed or not.

If you compare the output with Figure [fig:storks](#fig:storks), you can see that
r<sup>2</sup> value is called "R-squared", the p-value is called "Prob
(F-statistic)", the y-intercept is the first value in the "Intercept"
row, the slope is the first value in the "Storks" row. You can also
extract these parameters from the model results object like this:



In [1]:
# retrieve values from the model results
slope   :float = results.params[1]  # the slope
y0      :float = results.params[0]  # the y-intercept
rsquare :float = results.rsquared   # rsquare
pvalue  :float = results.pvalues[1] # the pvalue

Using these parameters, we can now calculate the regression line using
the `predict()` method of the model results object and add it to the
graph.  We will explore how to create the confidence interval in the
next module.



### References



-   Robert Matthews, Storks Devilver Babies (p = 0.008), Teaching
    Statistics 22:2, p 36-38, 2000,
    [https://doi.org/10.1111/1467-9639.00013](https://doi.org/10.1111/1467-9639.00013)

